In [ ]:
import pandas as pd

In [ ]:
CSV_FILE_PATH = "scores/scores_updated.csv"
try:
    df = pd.read_csv(CSV_FILE_PATH)
except FileNotFoundError:
    print(f"ERRO: O arquivo '{CSV_FILE_PATH}' não foi encontrado.")
    print("Por favor, verifique se o script está na mesma pasta que o arquivo CSV.")
    exit()

In [ ]:
# --- 1. ANÁLISE PARA A SEÇÃO "EXPERIMENTOS" ---
# Métrica: Média dos melhores scores individuais de cada imagem.

# Encontra a linha com o maior 'DiademScore' para cada combinação de Filtro e Imagem
best_individuals = df.loc[df.groupby(["FilterType", "ImageID"])["DiademScore"].idxmax()]

print("\n--- Melhores Scores Individuais por Filtro (Dataset Completo) ---\n")
for f_type in ["frangi", "kumar", "yang"]:
    print(f"Filtro: {f_type.capitalize()}")

    table_data = best_individuals[best_individuals["FilterType"] == f_type]

    # Seleciona as colunas relevantes para cada filtro
    if f_type == "yang":
        cols = [
            "ImageID",
            "DiademScore",
            "SigMin",
            "SigMax",
            "SigStep",
            "NeuronThreshold",
            "PruningThreshold",
            "SmoothingFactor",
        ]
    else:
        cols = [
            "ImageID",
            "DiademScore",
            "SigMin",
            "SigMax",
            "SigStep",
            "PruningThreshold",
            "SmoothingFactor",
        ]
    print(table_data[cols].sort_values(by="ImageID").to_csv(index=False))

In [ ]:
# Calcula a média desses melhores scores
avg_of_bests_full = best_individuals.groupby("FilterType")["DiademScore"].mean().reset_index()

In [ ]:
# Análise para o dataset filtrado
df_filtered = df[~df["ImageID"].isin(["OP_3", "OP_5", "OP_8"])]
best_individuals_filtered = df_filtered.loc[
    df_filtered.groupby(["FilterType", "ImageID"])["DiademScore"].idxmax()
]
avg_of_bests_filtered = (
    best_individuals_filtered.groupby("FilterType")["DiademScore"].mean().reset_index()
)

In [ ]:
# Junta os resultados para a tabela final da seção
summary_table_exp = pd.merge(
    avg_of_bests_full.rename(columns={"DiademScore": "Score médio (dataset completo)"}),
    avg_of_bests_filtered.rename(columns={"DiademScore": "Score médio (dataset filtrado)"}),
    on="FilterType",
)

In [ ]:
print("\n--- Tabela Resumo para 'Experimentos' (Média dos Melhores Scores) ---\n")
print(summary_table_exp.to_csv(index=False))

In [ ]:
# --- 2. ANÁLISE PARA A SEÇÃO "RESULTADOS E DISCUSSÃO" ---
# Métrica: Melhor média de score obtida por uma única configuração de parâmetros.
print("Análise para a Seção 'Resultados e Discussão'")
parameter_cols = [
    "FilterType",
    "SigMin",
    "SigMax",
    "SigStep",
    "NeuronThreshold",
    "PruningThreshold",
    "SmoothingFactor",
]

# Análise no Dataset Completo
avg_per_run_full = df.groupby(parameter_cols)["DiademScore"].mean().reset_index()
best_avg_runs_full = avg_per_run_full.loc[
    avg_per_run_full.groupby("FilterType")["DiademScore"].idxmax()
]

print("\n--- Melhor Média por Configuração (Dataset Completo) ---\n")
print(best_avg_runs_full.to_csv(index=False))

In [ ]:
# Análise no Dataset Filtrado
avg_per_run_filtered = df_filtered.groupby(parameter_cols)["DiademScore"].mean().reset_index()
best_avg_runs_filtered = avg_per_run_filtered.loc[
    avg_per_run_filtered.groupby("FilterType")["DiademScore"].idxmax()
]

print("\n\n--- Melhor Média por Configuração (Dataset Filtrado) ---\n")
print(best_avg_runs_filtered.to_csv(index=False))